In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
# NIH data
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)


#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
#print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

from config2 import *

from base2 import (width, height, radius, calc_target_coordinates_centered,
                  calc_rad_angle_from_coordinates)

import numpy as np
target_angs = (np.array([157.5, 112.5, 67.5, 22.5]) + 90) * \
              (np.pi/180)
target_coords = calc_target_coordinates_centered(target_angs)

In [ ]:
#assert sns.__version__ == '0.12.2', sns.__version__

In [ ]:
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

tgtc = ['b','g','r','brown']

# init

## per subj sep

In [ ]:
from behav_proc import *

behav_df_all = []
#or subj in subjects[:4]:#[si]
for subj in subjects: #[:1]:#[si]
    behav_data_dir = pjoin(data_dir_input,'behavdata')
    #behavdata
    task = 'VisuoMotor'
    fname = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df.pkl' )
    # where we would save cleaned df
    fname_EC = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df_EC.pkl' )
    behav_df_full = pd.read_pickle(fname)

    behav_df = pd.read_pickle(fname)
    behav_df['subject'] = subj
    addBehavCols(behav_df)
    
    behav_df_all += [behav_df]
    
behav_df_all = pd.concat(behav_df_all)
# behav_df_all['correction'] = None
# behav_df_all['err_sens'] = None
df_all = behav_df_all.reset_index()

In [ ]:
behav_df.groupby('block_name')['trials'].min().to_dict()

In [ ]:
behav_df.duplicated(['subject','trials']).any()

## together

In [ ]:
from behav_proc import *
#updstr = ''
updstr = '_upd'
behav_df_all = []
#or subj in subjects[:4]:#[si]
for subj in subjects:
    behav_data_dir = pjoin(data_dir_input,'behavdata')
    #behavdata
    task = 'VisuoMotor'
    fname = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df{updstr}.pkl' )
    # where we would save cleaned df
    #fname_EC = pjoin(path_data, subj, 'behavdata',
    #                f'behav_{task}_df_EC.pkl' )
    behav_df_full = pd.read_pickle(fname)

    behav_df = pd.read_pickle(fname)
    behav_df['subject'] = subj
    
    behav_df_all += [behav_df]
    
behav_df_all = pd.concat(behav_df_all, ignore_index=1).drop(columns=['level_0'])
# behav_df_all['correction'] = None
# behav_df_all['err_sens'] = None
df_all = behav_df_all.reset_index()

In [ ]:
assert len( df_all['subject'].unique() ) == 20

In [ ]:
# fn = 'df_all_11.01.24.pkl.zip'
# behav_df_all.to_pickle(pjoin(path_data,fn) , compression='zip')
# behav_df_all.to_pickle(pjoin('/home/demitau/current/merr_data',fn) , compression='zip')

In [ ]:
#%debug
import sys, traceback
try:
    addBehavCols(df_all)
except Exception as e:
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    
    
    stackframe = exc_info[2].tb_next.tb_frame
    local_vars_in_fun = stackframe.f_locals
    display(exc_info, stackframe)
    print('line num = ',exc_info[2].tb_next.tb_lineno)


In [ ]:
type(local_vars_in_fun['dfc']['target_locs'].iloc[0])

In [ ]:
checkErrBounds(df_all)

In [ ]:
#from behav_proc import adjustErrBoundsPi
#adjustErrBoundsPi(df_all,['error'])
#checkErrBounds(df_all)

## Continue init

In [ ]:
pertvals = list( df_all['perturbation'].unique() )
pertvals = list(pertvals) + [None]

envs = ['stable','random','all']
#envs = ['stable','random']
tgt_inds_all = list( df_all['target_inds'].unique() )  + [None]
#tgt_inds_all = list( df_all['target_inds'].unique() )  
#tgt_inds_all = [None]
#tgt_inds_all = [1]
nsubj = df_all['subject'].nunique()

#subj_list = subjects[:2]
#subj_list = ['mean'] 
#subj_list = subjects[:2] + ['mean']
subj_list = subjects

respect_trial_inds = True
ww = 6; hh = 3
nr = len(subj_list) * len(pertvals)
nc = len(envs)

from figure.plot_behav2 import *
# for envi,env in enumerate(envs):
#     for pertv in pertvals:
#         if (env,pertv) not in allowed_env_pert_combin:
#             print('skipping',env,pertv)
#         for tgti in tgt_inds_all:
#             coln = getColn(pertv,tgti,env)
#             #print(coln)
#             #print(len(df_all[coln]))
#             df_all[coln]    = [np.nan] * len(df_all)

#### supp plots

In [ ]:
sns.pairplot(data=df_all, vars=['trials','RT'])

In [ ]:
bn2trial_st = {'stable1': [0, 24, 84, 108, 168, 192], 'stable2': [384, 408, 468, 492, 552, 576]}

In [ ]:
df_all['pert_stage_wb']

In [ ]:


df_all['trialwpert_wb'] = df_all.apply(f,1)


In [ ]:
#subjects =df_all['subject'].unique()
df = df_all
dfc = df.query('subject == @subjects[0]')


#dfc.plot?

for col in ['trialwpert_wb', 'trialwtgt_wpert_wb', 'trialwpertstage_wb',
           'trialwtgt_wpertstage_wb', 'trialwtgt_wpertstage_we']:
    if 'wtgt' in col:
        df_ = dfc.query('target_inds == 0')
    else:
        df_ = dfc
    #plt.figure(figsize=(13,3))
    ax = df_.plot(x='trials',y=[col,'pert_stage_wb'], figsize=(10,3))
    ax.set_title(col)
    ax.legend(loc='upper left')
    
    

In [ ]:
behav_df['trials'].max()

In [ ]:
local_vars_in_fun['bn2trial_st']

In [ ]:
tcn = local_vars_in_fun['tcn']
print(local_vars_in_fun['tmax'], tcn)
local_vars_in_fun['df_all'][tcn].max()

In [ ]:
#df_all.to_pickle( pjoin(path_fig, 'output_intermed.gzip'),'gzip')

In [ ]:
for col in ['trialwpert_wb', 'trialwtgt_wpert_wb', 'trialwpertstage_wb',
           'trialwtgt_wpertstage_wb', 'trialwtgt_wpertstage_we']:
    for pert in [0.,30,-30.]:
        mx = df_all.query('perturbation == @pert')[col].max()
        print(f'{col:20}, pert={pert:3.0f},   mx={mx:5.0f}')

In [ ]:
print( df_['pert_stage_wb'].unique() )

In [ ]:
df_['pert_stage_wb'].astype(int)

In [ ]:
df_['block_name'].unique(), df_['perturbation'].unique()

In [ ]:
sns.lineplot?

In [ ]:
sum(c)

In [ ]:
df_ = df_all[~df_all['pert_stage_wb'].isna() ].copy()#.query('trial_shift_size == 1')
df_['pert_stage_wb'] = df_['pert_stage_wb'].astype(int)#.astype('category')
#df_ = df_.query('pert_stage_wb in [1,3]')
df_ = df_.query('pert_stage_wb in [1,3]')
print( df_['pert_stage_wb'].unique() )
sns.lineplot(data=df_, y ='error_pscadj_pertstageadj', 
             #x = 'trialwpertstage_wb',
             x = 'trialwpertstage_wb',
            hue = 'pert_stage')

In [ ]:
df_ = df_all[~df_all['pert_stage_wb'].isna() ].copy()#.query('trial_shift_size == 1')
df_['pert_stage_wb'] = df_['pert_stage_wb']#.astype('category')
df_ = df_.query('pert_stage_wb in [1,3]')
sns.lineplot(data=df_, y ='error_pscadj_pertstageadj', x = 'trialwtgt_wpertstage_wb',
            hue = 'pert_stage')

In [ ]:
df_ = df_all[~df_all['pert_stage_wb'].isna() ].copy()#.query('trial_shift_size == 1')
df_['pert_stage_wb'] = df_['pert_stage_wb']#.astype('category')
df_ = df_.query('pert_stage_wb in [0,2,4]')
#df_['pert_stage_wb'] = df_['pert_stage_wb'].astype('category')
print( df_['pert_stage_wb'].unique() )
sns.lineplot(data=df_, y ='error_pscadj_pertstageadj', x = 'trialwpertstage_wb',
            hue = 'pert_stage')
# sns.relplot(data=df_, kind='line', y ='error', x = 'trialwpertstage_wb',
#             row = 'pert_stage_wb')

In [ ]:
df_ = df_all[~df_all['pert_stage_wb'].isna() ].copy()#.query('trial_shift_size == 1')
df_['pert_stage_wb'] = df_['pert_stage_wb']#.astype('category')
df_ = df_.query('pert_stage_wb in [0,2,4]')
sns.lineplot(data=df_, y ='error_pscadj_pertstageadj', x = 'trialwtgt_wpertstage_wb',
            hue = 'pert_stage')

In [ ]:
df_all['trialwpert_wb'].max()

In [ ]:
df_all['trialwtgt_wpert']

In [ ]:
from error_sensitivity import computeErrSens2



In [ ]:
behav_df

# Calc err sens in different ways

#### debug

In [ ]:
len(err_sens_nh) == len(trial_inds_nh)

In [ ]:
df_all.loc[trial_inds_nh,'err_sens_perenv']

In [ ]:
np.any(~np.isnan(df_all['err_sens_perenv'] ) )

In [ ]:
df_all.loc[trial_inds_nh,'err_sens_perenv'] = df_esv_nh['err_sens']

In [ ]:
df_all.loc[trial_inds_nh,'err_sens_perenv'] = err_sens_nh

In [ ]:
df_all.loc[trial_inds_nh,'err_sens_perenv']

In [ ]:
help( np.mean )

In [ ]:
df_all

In [ ]:
df[['trials',coln]]#.groupby('trials').count()

In [ ]:
tgti

In [ ]:
print(env,subj,pertv,tgti)

In [ ]:
envcode

In [ ]:
df = df_all
if pertv is not None:
    df = df[df['perturbation'] == pertv]  
    
print (len(df))
if tgti is not None:
    df = df[df['target_inds'] == tgti] 
print (len(df))    

if env in env2envcode:  # make trial inds (within subject)
    envcode = env2envcode[env]
    coln = 'err_sens_perenv'
    db_inds = np.where(df['environment'] == envcode)[0]                    
    df = df[df['environment'] == envcode]
else:
    db_inds = np.arange(len(environment)) 
    coln = 'err_sens_all'
    
df

In [ ]:
#err_sens_nh   = grpme[coln]
err_sens_nh

In [ ]:
plt.plot(err_sens_nh)

In [ ]:
grp.count()

In [ ]:
 print(env,subj,pertv,tgti)

In [ ]:
a,b = list(grp)[0] 
a,b

In [ ]:
df_all.groupby

In [ ]:
#df.values[:,1]

In [ ]:
display( grp.apply(lambda g: g.mean(skipna=True)) )
grp.apply(lambda g: av(g,coln))#.reset_index()

In [ ]:
#grp.get_group(5)#; print(coln)
av( grp.get_group(5), coln)

In [ ]:
#grpme.values

#### final

In [ ]:
import gc; gc.collect()

In [ ]:
pd.options.mode.use_inf_as_na = True

In [ ]:
DEBUG = 0

##### debug subjStat

In [ ]:
from figure.plot_behav2 import subjStat
#trialcol = 'trialwb'
# trialcol = 'trialwe'
# pertv = None
# tgti = 0
# env = 'stable' 

env,pertv,gseqc,tgti,trial_group_col = ('all', None, (0,), 3, 'trials')
coln = getColn(pertv, tgti, env, trial_group_col)
print(coln)
df = getSubDf(df_all,'mean',pertv, tgti, env, False, pert_seq_code=gseqc[0])

#print(df['subject'].unique(), df['environment'].unique(), df['perturbation'].unique(), df['target_inds'].unique() )
from collections import Counter

print( Counter(df['subject']), Counter(df['environment']), 
      Counter(df['perturbation']), Counter(df['target_inds']) )


grpstat = subjStat(df, coln, trial_group_col)
err_sens_nh   = grpstat[coln]
trial_inds_nh = grpstat[trialcol]
display(grpstat)
#display(grpstd)

In [ ]:
df.subject.unique()

In [ ]:
df.columns[:25]

In [ ]:
cols = ['subject', 'environment', 'perturbation',
        'trials', 'trialwb', 'trialwe', 'err', 'RT']  + [coln]
df.loc[ df['trialwb'] == 0,  cols]

In [ ]:
plt.plot( df['perturbation'] )
plt.plot(df['trialwb'] / 10)
plt.plot(df['trialwe'] / 20)

#### fill per subject table

In [ ]:
#df_all.loc[:50,['environment','target_inds','perturbation','trials','trialwtgt','trialwtgt_wpert_we','trialwtgt_wpert_wb']]

In [ ]:
df_all.columns

In [ ]:
allowed_env_pert_combin = [('all',0.), ('all',30.), ('all',-30.), ('all',None), 
                           ('stable',0.),('stable',30.),('stable',-30.),('stable',None),('random',0.),
                           ('random',None)]

allowed_env_trialcol_combin = [('all','trials'), ('all','trialwtgt'), 
        ('all','trialwtgt_wpert_wb'), ('all','trialwtgt_wpert_we'), 
        ('stable','trialwtgt'), ('stable','trialwtgt_we'), ('random','trialwe'),
        ('stable','trialwpert'), ('stable','trialwe'), ('stable','trialwb'), 
        ('stable','trialwpert_we'), ('stable','trialwpert_wb'),  
        ('stable', 'trialwtgt_wpert_wb') , ('stable','trialwtgt_wpert_we')]

allowed_env_trialcol_combin = [('stable','trialwe'), ('stable','trialwtgt_we'),
('random','trialwe'), ('random','trialwtgt_we')]
#  -- meaningless if separate across tgt because targets are not sync across participants

#allowed_env_trialcol_combin = [('all','trials'), 
#                                 ('stable','trialwe'), ('random','trialwe') ]

allowed_tgt_trialcol_combin = [(None,'trialwe'), (None,'trials'), 
                               (None,'trialwpert_wb'), (None,'trialwpert_we')]
for tgti in range(4):  # only numeric, not None here
    allowed_tgt_trialcol_combin += [(tgti,'trialwtgt')] + [(tgti,'trialwe')]
    allowed_tgt_trialcol_combin += [(tgti,'trialwtgt_we')] 
    allowed_tgt_trialcol_combin += [(tgti,'trialwpert_wb')] 
    allowed_tgt_trialcol_combin += [(tgti,'trialwpert_we')]
    allowed_tgt_trialcol_combin += [(tgti,'trialwtgt_wpert_wb')]
    allowed_tgt_trialcol_combin += [(tgti,'trialwtgt_wpert_we')]
    allowed_tgt_trialcol_combin += [(tgti,'trialwb')] 
    
#allowed_env_tgt_trialcol_combin = ['stable',]

In [ ]:
allowed_envblock = [('stable','stable1'), ('stable','stable2'), 
                   ('stable','all'), ('random','random1'), ('random','random2'),
                   ('random','all'), ('all','all')]

In [ ]:
block_names_to_calc = block_names + ['all']

In [ ]:
DEBUG = 0
subj_list = subjects

In [ ]:
pertvals_to_calc = pertvals + [None]
pertvals_to_calc

#### oldv (adding columns)

In [ ]:
from figure.plot_behav2 import getSubDf, getColn
error_type = 'MPE'  # observed - goal, motor performance error
colns_set  = []; colns_skip = [];
debug_break = 0
for env, trial_group_col in allowed_env_trialcol_combin:
    for block_name in block_names_to_calc:
        if (env,block_name) not in allowed_envblock:
            continue
    for subj in subj_list:#[si]
        for pertv in pertvals_to_calc:
            if (env,pertv) not in allowed_env_pert_combin:
                print('skipping',env,pertv)
                continue
            for tgti in tgt_inds_all:
                if (tgti,trial_group_col) not in allowed_tgt_trialcol_combin:
                    continue
                print(env,subj,pertv,tgti,trial_group_col)
                #df = df_all
                # if we take only non-hit then, since we'll compute err sens sequentially 
                # we'll get wrong
                #if trial_group_col in ['trialwb']:
                #    raise ValueError('not implemented')
                df = getSubDf(df_all, subj, pertv,tgti,env,block_name,
                              non_hit = False)
                db_inds = df.index                
                
                coln = getColn(pertv, tgti, env, block_name, None) #, trial_group_col)        
                print('  ',coln,len(df))                                
                if (len(df) == 0) or (len(db_inds) == 0):
                    #rowi += 1
                    print('skip',coln,subj)
                    colns_skip += [coln]
                    if DEBUG:
                        debug_break = 1
                        break
                    continue                      
                                            
                # resetting index is important
                r = computeErrSens2(df.reset_index(), df_inds=None, 
                                    error_type=error_type)
                nhna, df_esv, ndf2vn = r            

                #df_esv_nh     = df_esv[df_esv['non_hit_not_adj'] & \
                #                       ~np.isinf(df_esv['err_sens'])]
                #err_sens_nh   = np.array( df_esv_nh['err_sens'] )
                #trial_inds_nh = np.array( df_esv_nh['trial_inds_glob'] )

                #pert_nh = df_esv_nh['perturbation']
                
                # if I don't convert to array then there is an indexing problem
                # even though I try to work wtih db_inds it assigns elsewhere 
                # (or does not assigne at all)
                es_vals = np.array( df_esv['err_sens'] ) 
                
                assert np.any(~np.isnan(es_vals)), coln  # at least one is not None
                assert np.any(~np.isnan(es_vals)), coln  # at least one is not None
                                
                colns_set += [coln]
                df_all.loc[db_inds,coln] = es_vals  # NOT _nh, otherwise different number
                df_all.loc[db_inds,'error_type'] = error_type                
                
                if DEBUG and coln == 'err_sens_pert-30_tgt3_envSep':                    
                    debug_break = 1; print('brk')
                    db_inds_save = db_inds
                    df_save = df
                    break
            if debug_break:
                break
        if debug_break:
            break
    if debug_break:
        break
print('Finished successfully')

#### newv (concat)

In [ ]:
#from figure.plot_behav2 import getSubDf, getColn, computeErrSensVersions
envs_cur = ['stable', 'random']
block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ (0,1) ]
tgt_inds_cur = tgt_inds_all
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
error_type = 'MPE'  # observed - goal, motor performance error

In [ ]:
# compute separately for variable shift sizes to generate grids later
# separately to avoid filtering problems later (I could get too many values per subj 
# due to multiple rows with diffrent trial shift size and same trial)
envs_cur = ['stable', 'random', 'all']
tgt_inds_cur = tgt_inds_all
#%debug
try:
    df_all_multi_tsz, ndf2vn = computeErrSensVersions(df_all, envs_cur,
        block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
        dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
        coln_nh = 'non_hit_not_adj',
        coln_nh_out = 'non_hit_shifted',
        computation_ver='computeErrSens3',
        subj_list = subjects, error_type=error_type,
        trial_shift_sizes = np.arange(1,5),
                 addvars=[], retention_factor=1)
    
    assert not df_all_multi_tsz.duplicated().any() 
except Exception as e:
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    
    ei = exc_info[2]    
    #display(ei.tb_frame)
    psf = ei

    lfprev = None; lf = None
    lfs = []
    di = 0
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame        

        psf = psf.tb_next  
        if ('conda' not in stackframe.f_code.co_filename) and\
            ('mamba' not in stackframe.f_code.co_filename):        
            display(di, stackframe)
            lfprev = lf
            lf = stackframe.f_locals
            lfs += [lf]
            di += 1
        if psf is None:
            break
        
    excsfmt = traceback.format_exc()
    print(excsfmt)




In [ ]:
df_ = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trials"')
assert not df_.duplicated(['subject','trials']).any()

In [ ]:
df__ = truncateDf(df_, q=0,infnan_handling='discard',coln='err_sens' )
#plt.figure()
me = df__.groupby(['subject','env'], observed=True).\
    mean(numeric_only=1).reset_index()
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='env',  order=['stable','random'], hue = 'env', palette = ['tab:grey','tab:orange'], legend=None)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)

In [ ]:
sns.relplot(data = df__, kind='line',x='trials',y='err_sens')

In [ ]:
df_ = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trials"')
assert not df_.duplicated(['subject','trials']).any()
sns.relplot(data = df_, kind='line',x='trials',y='error_pscadj')

In [ ]:
10 - -10

In [ ]:
df_all.query('subject == "sub04_XNDMUSRS" and trials in [305,306]')\
[['subject','trials','error','target_locs', 'org_feedback']]

In [ ]:
df_all_multi_tsz.query('correction > 1.9')\
    [['subject','trials','error','prev_error','target_locs','correction','trial_shift_size']]

In [ ]:
checkErrBounds(df_all)

In [ ]:
checkErrBounds(df_all_multi_tsz)

In [ ]:
lf['df']

In [ ]:
df__ = local_vars_in_fun['df']

In [ ]:
df__.drop(columns=['index'])

In [ ]:
df_all_multi_tsz['non_hit_not_adj']

##### supp

Q: non hit -- do I have to calc them
    separate when I compute for primary vars and for err sens?
    I mean, to reduce doing AND for current and prev trials
Q: should I compute multiple trial groupings or just one at a time?
TODO: behav table within pert
when do fit maybe remove outliers

should I keep merged and non-merged pert in the same fun of dec err sens?
it is better for debuggging but less good for coding

TODO: find code where I was extracting locals from traceback

##### dbg

In [ ]:
for s in [1,2]:
    dfdbg = df_all_multi_tsz_.query('(trial_shift_size == @s) and (trial_group_col_calc == "trialwe")')
    cols = ['trials', 'target_inds', 'target_locs',
                      'trial_shift_size','dist_rad_from_prevtgt',
                      'dist_rad_from_tgt-1','dist_rad_from_tgt-2',
                      'dist_rad_from_prevtgt2']
    r = dfdbg.loc[dfdbg.index[:10],cols]
    display(r)

In [ ]:
help(df_all.apply)

In [ ]:
envs_cur =  ['all']
tgt_inds_cur = [None]
df_all4 = computeErrSensVersions(df_all, envs_cur,block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
               dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
                                subj_list = subjects[:1], error_type=error_type)

#### compare with Romain direct ver

In [ ]:
%run -i ../figure/plot_es_q.py

In [ ]:
subjects[0],all_es_stable2[0]

In [ ]:
all_es[0][:20]

In [ ]:
df_all4.query('subject == "sub01_WGPOZPEE"')[['trials','err_sens']]

In [ ]:
len(es)

In [ ]:
corr = (Y[0]-Y[2]) - (Y[1]-Y[3])
non_hit[:20], es[:20], Y[4][:20], corr[:20] 

In [ ]:
a = np.vstack([df_esv['err_sens'].values, es])
np.isnan(a[0]) ^ np.isnan(a[1])

In [ ]:
#### dbg
subj = 'sub01_WGPOZPEE'
env = 'all'
df = getSubDf(df_all, subj, pertv,tgti,env,block_name,
              non_hit = False, verbose=1)
db_inds = df.index

tgn = getTrialGroupName(pertv, tgti, env, block_name)
coln = getColn(pertv, tgti, env, block_name, None) #, trial_group_col)
print('  ',tgn, coln,len(df))

# resetting index is important
r = computeErrSens2(df.reset_index(), df_inds=None,
                    error_type=error_type,
                    colname_nh = 'non_hit_shifted',
                    correct_hit = 'inf', shiftsz=2, err_sens_coln = 'err_sens')
nhna, df_esv, ndf2vn = r
df_esv

dfcur = df.copy()

dfcur['dist_rad_from_prevtgt2'] = dfcur['target_locs'].values -\
    df_esv['prev_target'].values
dfcur['dist_rad_from_prevtgt2'] =\
    dfcur['dist_rad_from_prevtgt2'].abs().apply(lbd, 1)
dfcur.drop(['trajectoryX','trajectoryY'],1)

In [ ]:
#np.nanmean(all_es,0)

In [ ]:
arr = 1. / np.array([np.inf])
#arr.dtype

In [ ]:
print(Y[4][:1])
arr = 1 / Y[4][:1]  # dividing by zero gives divides by zero error
arr, type(arr), arr.dtype

In [ ]:
 (~df_all4[['trials']].notna()).sum()

In [ ]:
df_all4[ ~df_all4['err_sens'].notna() ]['error']

In [ ]:
df_all4.loc[:20,['trials','err_sens','non_hit_not_adj','non_hit_shifted', 'error']]

In [ ]:
#np.zeros(1) / ( np.ones(1) * np.inf) 

In [ ]:
cols = ['subject','trials','trialwe','trialwtgt_we','target_inds',
               'error','correction','non_hit_not_adj','non_hit_shifted']

In [ ]:
df_all2.columns

In [ ]:
#df_all2.sort_values(by=['subject','trials'])

In [ ]:
dfc = df_all2[cols + ['trial_group_col_calc','err_sens','error']]

In [ ]:
df_all2['trial_group_col_calc'].unique(), df_all3['trial_group_col_calc'].unique(), 

In [ ]:
dfc[dfc['trial_group_col_calc'] == 'trialwtgt_we']

In [ ]:
dfc[dfc['trial_group_col_calc'] != 'trialwtgt_we']

In [ ]:
dfc['trial_group_col_calc'].unique()

In [ ]:
import gc; gc.collect()

In [ ]:
dfc = df_all[df_all['subject'] == subj]
#dfc['perturbation']

#### fill mean table

In [ ]:
allowed_env_trialcol_combin

In [ ]:
# allowed_pert_gseqc = [('None', 'all'), ('None', 'sep'), ('0', 'all'), ('0', 'sep'),
#                       ('30','sep'), ('-30','sep')]
allowed_pert_gseqc = [(None, [0,1]), (None, [0]),  (None, [1]),
                      (0, [0,1]), (0, [0]), (0, [1]),
                      (30,[0]), (30,[1]), (-30,[0]),  (-30,[1])]
allowed_pert_gseqc = [ (s,tuple(a) ) for (s,a) in allowed_pert_gseqc ]
display(allowed_env_trialcol_combin, allowed_env_pert_combin, 
        allowed_pert_gseqc, tgt_inds_all, pertvals)
print(allowed_tgt_trialcol_combin)

#### oldv

In [ ]:
# compute mean veersion  
# not then if I set come value to None and others to float, 
# then None gets converted to NaN
dfme = []
dfme2 = []
#for envi,env in enumerate(envs):
nrows = []
for env, trial_group_col in allowed_env_trialcol_combin:
    for block_name in block_names_to_calc:
        if (env,block_name) not in allowed_envblock:
            continue
        rowi = 0    
        for pertv in pertvals:
            for gseqc in [(0,), (1,), (0,1)]:
                if (pertv,gseqc) not in allowed_pert_gseqc:
                    print('skipping',pertv,gseqc)
                    continue                
                if (env,pertv) not in allowed_env_pert_combin:
                    print('skipping',env,pertv)
                    continue            

                for tgti in tgt_inds_all:
                    if (tgti,trial_group_col) not in allowed_tgt_trialcol_combin:
                        continue
                    print(env,pertv,gseqc,tgti,trial_group_col)
                    subj = 'mean'
                    pert_seq_code = None
                    if gseqc != (0,1):
                        pert_seq_code = gseqc[0]
                    df = getSubDf(df_all, subj, pertv,tgti,env, non_hit = False,
                                 pert_seq_code = pert_seq_code)
                    db_inds = df.index

                    coln_calc = getColn(pertv, tgti, env, None)
                    assert coln in colns_set
                    if (len(df) == 0) or (len(db_inds) == 0):
                        #rowi += 1
                        continue                      
                    #%debug
                    #ax = axs[rowi,envi]
        #             try:

                    coln_av = getColn(pertv, tgti, env, trial_group_col)
                    #
                    grpstat = subjStat(df, coln_calc, trial_group_col)
                    #pscstr = ','.join( list( map(str, list( sorted(df['pert_seq_code'].unique() ) ) ) ) )
                    pscstr = ','.join( list( map(str, gseqc ) ) )    
                    # some things are for filtering, some for actual usage,
                    # that's why pertrubation is pertv and pert_nh is actual vals
                    r = { 'environment':env, 'trial_group_col':trial_group_col, 
                         'perturbation':pertv, 'pert_seq_code':pscstr,
                         'pert_nh':grpstat['perturbation'],
                         'target_inds':tgti,  'nav_nh': grpstat['nav'],
                         'trial_inds_nh':grpstat[trial_group_col], 
                         'err_sens_me_nh':grpstat['mean'], 
                         'err_sens_std_nh':grpstat['std'], 
                         'err_sens_sem_nh':grpstat['sem'],
                         'coln':coln_av, 'coln_calc':coln_calc  }

                    r = { 'environment':env, 'trial_group_col':trial_group_col, 
                         'perturbation':pertv, 'pert_seq_code':pscstr,
                         'pert_nh':grpstat['perturbation'],
                         'target_inds':tgti,  'nav_nh': grpstat['nav'],
                         'trial_inds_nh':grpstat[trial_group_col], 
                         'err_sens_me_nh':grpstat['mean'], 
                         'err_sens_std_nh':grpstat['std'], 
                         'err_sens_sem_nh':grpstat['sem'],
                         'coln':coln_av, 'coln_calc':coln_calc  }
                    dfme += [r]
                    rowi += 1
        nrows += [rowi]
nrows = np.max(nrows); print(nrows)
dfme = pd.DataFrame(dfme)
dfme

#### newv

In [ ]:
df_all.columns

In [ ]:
*(tpl[:-1]), tgcalc,tgav

In [ ]:
from figure.plot_behav2 import computeStat

In [ ]:
#help( pd.concat)

In [ ]:
# compute mean veersion  
# not then if I set come value to None and others to float, 
# then None gets converted to NaN
envs_cur = ['stable', 'random']
block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ (0,1) ]
tgt_inds_cur = [None]
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
trgcol_pairs = [('trialwe','trialwe'), ('trialwtgt_we','trialwtgt_we')]

dfme = []
from itertools import product as itprod
p = itprod(envs_cur,block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
           dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,trgcol_pairs)
for tpl in p:
    #print(len(tpl), tpl)
    (env,block_name,pertv,gseqc,tgti,drptgt,dtptgt,trgcolp) = tpl
    (tgcalc,tgav)= trgcolp
    
    r = computeStat(df_all,*(tpl[:-1]), tgcalc,tgav, verbose=0  )
    if r is None:
        print()
        break
        #continue
    grpstat, coln_calc, coln_av  = r
    pscstr = ','.join( list( map(str, gseqc ) ) )    
    # some things are for filtering, some for actual usage,
    # that's why pertrubation is pertv and pert_nh is actual vals
#     r = { 'environment':env, 'trial_group_col':trial_group_col, 
#          'perturbation':pertv, 'pert_seq_code':pscstr,
#          'pert_nh':grpstat['perturbation'],
#          'target_inds':tgti,  'nav_nh': grpstat['nav'],
#          'trial_inds_nh':grpstat[trial_group_col], 
#          'err_sens_me_nh':grpstat['mean'], 
#          'err_sens_std_nh':grpstat['std'], 
#          'err_sens_sem_nh':grpstat['sem'],
#          'coln':coln_av, 'coln_calc':coln_calc  }
    #dfme += [r]
    dfme += [grpstat] 
    
assert len(dfme)
dfme  = pd.concat(dfme)
dfme.reset_index(inplace=True)
dfme

In [ ]:
dfme

In [ ]:
df_all.columns

In [ ]:
## reshape df_all
df_all2 = df_all.copy()
#df_all['err_sens'] = 

# save

In [ ]:
coln = 'err_sens'
# old
# dfc  = truncateDf(df_all2, coln, q=0.05, clean_infnan=True)
# dfc0 = truncateDf(df_all2, coln, q=None, clean_infnan=True)

# dfc_  = truncateDf(df_all3, coln, q=0.05, clean_infnan=True)
# dfc0_ = truncateDf(df_all3, coln, q=None, clean_infnan=True)

dfc_multi_tsz  = truncateDf(df_all_multi_tsz, coln, q=0.05, infnan_handling='discard')
dfc0_multi_tsz  = truncateDf(df_all_multi_tsz, coln, q=0.0, infnan_handling='discard')

qs = 'trial_shift_size == 1'
dfc  = dfc_multi_tsz.query(qs)
dfc0 = dfc0_multi_tsz.query(qs)

In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz.pkl.zip')
print(fnf)

In [ ]:
fn = 'df_all_multi_tsz.pkl.zip'
fnf = pjoin(path_data,fn)
print(fnf)
df_all_multi_tsz.to_pickle(fnf, 
                           compression='zip')
df_all_multi_tsz.to_pickle(pjoin('/home/demitau/current/merr_data',fn), 
                           compression='zip')

dfc_multi_tsz.   to_pickle(pjoin(path_data,'df_all_multi_tsz_trunc_q=0.05.pkl.zip'),
                          compression='zip')

# Plot pooled across time [violins comparable with Romain]

In [ ]:
# def f(row):
#     bn = row['block_name']
#     ps = row['pert_stage_wb']
#     if np.isnan(ps):
#         return None
#     if bn.endswith('2'):
#         ps = int(ps) + 5

#     return ps
# df_all_multi_tsz['pert_stage'] = df_all_multi_tsz.apply(f,1)


In [ ]:
path_data

In [ ]:
#dfc_notna_multi_tsz = dfc_multi_tsz[~dfc_multi_tsz['err_sens'].isna()]

In [ ]:
len(df_all_multi_tsz), len(dfc_multi_tsz)

#### means per subj distr

In [ ]:
dfc_multi_tsz.columns

In [ ]:
#dfc['environment'].unique()

In [ ]:
dfc0

In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz.pkl.zip')
df_all_multi_tsz = pd.read_pickle(fnf)
coln = 'err_sens'
dfc_multi_tsz  = truncateDf(df_all_multi_tsz, coln, q=0.05, infnan_handling='discard')
dfc0_multi_tsz  = truncateDf(df_all_multi_tsz, coln, q=0.0, infnan_handling='discard')

qs = 'trial_shift_size == 1'
dfc  = dfc_multi_tsz.query(qs)
dfc0 = dfc0_multi_tsz.query(qs)

In [ ]:
qs = 'trial_shift_size == 1'
dfc  = dfc_multi_tsz.query(qs)
dfc0 = dfc0_multi_tsz.query(qs)

In [ ]:
dfc0.duplicated(['subject','trials','environment','trial_group_col_calc']).any()

In [ ]:
df_all_multi_tsz = pd.read_pickle('/home/demitau/data_Quentin/full_experiments/data2/df_all_multi_tsz.pkl')
#df_all_multi_tsz = pd.read_pickle(fnf)
coln = 'err_sens'
dfc_multi_tsz  = truncateDf(df_all_multi_tsz, coln, q=0.05, infnan_handling='discard')
dfc0_multi_tsz  = truncateDf(df_all_multi_tsz, coln, q=0.0, infnan_handling='discard')

qs = 'trial_shift_size == 1'
#dfc  = dfc_multi_tsz.query(qs)
dfc0 = dfc0_multi_tsz.query(qs)

In [ ]:
%matplotlib inline
me = dfc0.groupby(['subject','environment','trial_group_col_calc']).\
    mean(numeric_only=1).reset_index()
sns.catplot(data=me, kind='violin', y='err_sens', x='environment', 
            col='trial_group_col_calc')

In [ ]:
# not truncated
me = dfc0.groupby(['subject','environment','trial_group_col_calc']).\
    mean(numeric_only=1).reset_index()
sns.catplot(data=me, kind='violin', y='err_sens', x='environment', 
            col='trial_group_col_calc')

# truncated
me = dfc.groupby(['subject','environment','trial_group_col_calc']).\
    mean(numeric_only=1).reset_index()
sns.catplot(data=me, kind='violin', y='err_sens', x='environment', 
            col='trial_group_col_calc')

In [ ]:
tgcc = 'trialwe'
for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    dfc_s = df[(df['trial_group_col_calc'] == tgcc) ]
    grp = dfc_s.groupby(['subject','environment',])
    me = grp.mean(numeric_only = 1)['err_sens'].reset_index()
    plt.figure()
    ax = sns.violinplot(data=me, y='err_sens', x='environment')
    ax.set_title(f'{tgcc} {truncs}')
    ax.set_ylim(-0.4,0.8)
    
tgcc = 'trials'
#for truncs, df in zip(['truncate q=5%','truncate no'],[dfc_, dfc0_]):
for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    dfc_s = df[(df['trial_group_col_calc'] == tgcc) ]
    grp = dfc_s.groupby(['subject','environment'])
    me = grp.mean(numeric_only = 1)['err_sens'].reset_index()
    plt.figure()
    ax = sns.violinplot(data=me, y='err_sens', x='environment')
    ax.set_title(f'{tgcc} {truncs}')
    ax.set_ylim(-0.4,0.8)

In [ ]:
#dfc[~(dfc[coln].isna() | np.isinf(dfc[coln]) )]
dfc

In [ ]:
#dfc_s['environment']

#### per subj

In [ ]:
coln = 'err_sens'
tgcc = 'trialwe'
dfc_s = dfc[dfc['trial_group_col_calc'] == tgcc]
sns.violinplot(data=dfc_s, y=coln, hue='environment', x='subject', split=1)

In [ ]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})